<a href="https://colab.research.google.com/github/viniciusrpb/116319_estruturasdedados/blob/main/irony_detection_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r "/content/drive/My Drive/sentiment_analysis/irony_detection_EN_taskA" "irony_detection_EN"
#!cp -r "/content/drive/My Drive/sentiment_analysis/irony_detection_pt2" "irony_detection_pt2"

In [ ]:
!pip install tensorflow-datasets
!pip install tensorflow
!pip install transformers==4.37.2
!pip install tensorflow_addons
!pip install keras-tuner --upgrade

In [17]:
from keras.models import Sequential,Model
from keras.layers import Dense,Embedding,Reshape,BatchNormalization,Input,LSTM,Bidirectional
from transformers import TFBertModel,AutoTokenizer,BertTokenizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import gensim
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import pandas as pd
import keras_tuner
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split()
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words)

def preprocess_tweets(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" ", sentence)
    #sentence = re.sub(r'[" "]+', " ", sentence)
    #sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = clean_stopwords_shortwords(sentence)
    sentence = re.sub(r'@\w+', '',sentence)
    return sentence

In [32]:
def sentence_bert_encode(df,tokenizer,max_len):

  input_ids = []
  attention_masks = []

  for i in range(len(df)):
      encoded = tokenizer.encode_plus(
        df[i],
        add_special_tokens=True,
        max_length=max_len,
        pad_to_max_length=True,
        #return_tensors = 'tf',
        return_attention_mask=True
      )

      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])

  return np.array( input_ids),np.array( attention_masks)

In [36]:
def build_PreTrainedModel_bert(hp):

    max_sentence_length = 32

    input_tokens = Input(shape=(max_sentence_length,),dtype="int32")
    attention_masks = Input(shape=(max_sentence_length,),dtype="int32")

    bert_model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")

    embedding = bert_model([input_tokens,attention_masks])

    #embedding = bert_model.layers[0](input_tokens, attention_masks)
    #last_hidden_states = embedding.last_hidden_state
    fcc = Dense(hp.Choice('num_neurons_lstm',[64,128,256,512,1024,2048]), activation = 'relu')(embedding[1])
    output_layer = Dense(2, activation='softmax')(fcc)
    model_bert = Model(inputs=[input_tokens,attention_masks], outputs=output_layer)

    '''lstm = LSTM(hp.Choice('num_neurons_lstm',[64,128,256,512,1024,2048]),activation="relu")(newReshape)
    newout = Dense(hp.Choice('num_neurons_dense',[64,128,256,512,1024,2048]),activation="relu")(lstm)
    preds = Dense(num_classes, activation='softmax')(newout)
    model = Model(inputs, preds)'''

    f1_score2 = tfa.metrics.F1Score(num_classes=2, average='macro',threshold=0.5)

    adam = Adam(hp.Choice('learning_rate',[0.01,0.001,0.0001,0.0005,0.00001,0.00005,0.000001]))
    model_bert.compile(loss='categorical_crossentropy',optimizer=adam,metrics=[f1_score2])

    return model_bert

In [23]:
def getBestModel(hyperparameters):

    max_sentence_length = 32

    input_tokens = Input(shape=(max_sentence_length,),dtype="int64")
    attention_masks = Input(shape=(max_sentence_length,),dtype="int64")

    bert_model = TFBertModel.from_pretrained("google-bert/bert-base-uncased")

    embedding = bert_model([input_tokens,attention_masks])

    fcc = Dense(64, activation = 'relu')(embedding[1])
    output_layer = Dense(2, activation='softmax')(fcc)
    model = Model(inputs=[input_tokens,attention_masks], outputs=output_layer)

    f1_score2 = tfa.metrics.F1Score(num_classes=2, average='macro',threshold=0.5)

    adam = Adam(learning_rate=0.000003)#hyperparameters.get('learning_rate'))

    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=[f1_score2])

    return model

In [10]:
path_train = "irony_detection_EN/SemEval2018-T3-train-taskA.txt"
path_valid = "irony_detection_EN/SemEval2018-T3-valid-taskA.txt"
path_test = "irony_detection_EN/SemEval2018-T3_gold_test_taskA_emoji.txt"

In [11]:
df_train = pd.read_csv(path_train, sep='\t',encoding="UTF-8",low_memory = False)
df_valid = pd.read_csv(path_valid, sep='\t',encoding="UTF-8",low_memory = False)
df_test = pd.read_csv(path_test, sep='\t',encoding="UTF-8",low_memory = False)

df_train.head()

,Tweet index,Tweet text,Label
0,2248,Beautiful day,1
1,156,@Moose_eBooks You seriously bought a giant TV ...,0
2,2831,@NewDay: .@JebBush considering a run for the @...,1
3,842,a bad game last night. Way to go Packers! htt...,0
4,20,"But @DarklightDave was trying to find us, and ...",1


In [12]:
df_train['Tweet text'] = df_train['Tweet text'].map(preprocess_tweets)
df_valid['Tweet text'] = df_valid['Tweet text'].map(preprocess_tweets)
df_test['Tweet text'] = df_test['Tweet text'].map(preprocess_tweets)

In [33]:
max_sentence_length = 32

bert_tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

train_input_ids,train_attention_masks = sentence_bert_encode(df_train['Tweet text'],bert_tokenizer,max_sentence_length)
valid_input_ids,valid_attention_masks = sentence_bert_encode(df_valid['Tweet text'],bert_tokenizer,max_sentence_length)
test_input_ids,test_attention_masks = sentence_bert_encode(df_test['Tweet text'],bert_tokenizer,max_sentence_length)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [44]:
df_train['Label'] = pd.Categorical(df_train['Label'])
y_train = df_train['Label'].cat.codes

df_valid['Label'] = pd.Categorical(df_valid['Label'])
y_valid = df_valid['Label'].cat.codes

df_test['Label'] = pd.Categorical(df_test['Label'])
y_test = df_test['Label'].cat.codes

num_classes = 2

y_train_enc = to_categorical(df_train['Label'],num_classes)
y_valid_enc = to_categorical(df_valid['Label'],num_classes)
y_test_enc = to_categorical(df_test['Label'],num_classes)

Hyperparameter Optimization

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10)

tuner = keras_tuner.RandomSearch(
                build_PreTrainedModel_bert,
                directory='/content/drive/MyDrive/irony',
                objective='val_loss',
                project_name='bert_simple',
                max_trials=5,
                overwrite=True)

tuner.search([train_input_ids,train_attention_masks],y_train_enc, epochs=20, validation_data=([valid_input_ids,valid_attention_masks],y_valid_enc),callbacks = [early_stop])

best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]

string = "The hyperparameter search is complete. The number of neurons in FCC layers is "+str(best_hps.get('num_neurons_dense'))+"\nThe optimal number of units in the lstm layer is "+str(best_hps.get('num_neurons_lstm'))+" and the optimal learning rate for the optimizer is "+str(best_hps.get('learning_rate'))+"."

sentença de exemplo codificada

In [50]:
df_train2 = pd.concat([df_train,df_valid],ignore_index=True)
df_train2['Label'] = pd.Categorical(df_train2['Label'])
y_train2 = df_train2['Label'].cat.codes

y_train_enc2 = to_categorical(y_train2,num_classes)

train_input_ids,train_attention_masks = sentence_bert_encode(df_train2['Tweet text'],bert_tokenizer,max_sentence_length)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [51]:
best_model = getBestModel({})

early_stop2 = EarlyStopping(monitor='loss', patience=3)

history = best_model.fit([train_input_ids,train_attention_masks],y_train_enc,
                          epochs=4,
                          batch_size=32,
                          callbacks = [early_stop2])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Data cardinality is ambiguous:
  x sizes: 3053, 3053
  y sizes: 2442
Make sure all arrays contain the same number of samples.

sentença de exemplo decodificada

In [48]:
y_prob = best_model.predict([test_input_ids,test_attention_masks])
y_pred = np.argmax(y_prob,axis=1)

25/25 [==============================] - 5s 80ms/step


Codificando os labels

In [49]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.48      0.60       473
           1       0.50      0.79      0.61       311

    accuracy                           0.61       784
   macro avg       0.64      0.64      0.61       784
weighted avg       0.67      0.61      0.60       784

